In [1]:
import numpy as np
import tensorflow as tf

In [2]:
def conv1d(x, w, p=0, s=1):
    w_rotate = np.array(w[::-1])
    x_padded = np.array(x)
    if p > 0:
        zero_pad = np.zeros(shape=p)
        x_padded = np.concatenate([zero_pad,
                                   x_padded,
                                   zero_pad])
        
    res = []
    for i in range(0, int(len(x)/s), s):
        res.append(np.sum(x_padded[i:i+w_rotate.shape[0]] * w_rotate))
        
    return np.array(res)

In [3]:
x = [1, 3, 2, 4, 5, 6, 1, 3]
w = [1, 0, 3, 1, 2]

print('Conv1d Implementation:', conv1d(x, w, p=2, s=1))

Conv1d Implementation: [ 5. 14. 16. 26. 24. 34. 19. 22.]


In [4]:
print('NumPy Results:', np.convolve(x, w, mode='same'))

NumPy Results: [ 5 14 16 26 24 34 19 22]
